In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import numpy as np
from nltk.corpus import stopwords

In [2]:
df_tickets = pd.read_csv('tickets.csv', sep=';', quotechar = '\'')

In [3]:
df_tickets.dropna(inplace=True)

In [4]:
df_tickets

,ownerid,fullDESCRIPTION
0,63602,"<body> <div> <p><i><span style=""font-family: C..."
1,129127,"<div style=""font-family: Calibri , Arial , Hel..."
2,146705,"<div> <p>Estimados, buenos días</p> <p> </p> <..."
3,72603,"<body> <div style=""font-family: Calibri , Aria..."
4,131404,"<div style=""font-family: Calibri , Arial , Hel..."
...,...,...
60479,165002,"<body> <div style=""color: rgb(33, 33, 33); bac..."
60480,122748,"<div>Buen día, favor su ayuda para verificar e..."
60481,165002,"<body> <div style=""font-family: Calibri, Arial..."
60482,129127,"<div><p class=""MsoNormal"">Estimados<br /></p><..."


In [5]:
df_tickets["fullDESCRIPTION"] = df_tickets['fullDESCRIPTION'].str.replace('<[^<]+?>',' ', regex=True).str.replace('&nbsp',' ', regex=True).str.replace('[^\w\s]',' ', regex=True).apply(str.lower).str.replace('estimados','').str.replace('estimado','').str.replace('\xa0','')

In [6]:
df_tickets

,ownerid,fullDESCRIPTION
0,63602,a la fecha no hemos recibido el lecto...
1,129127,personal de soporte soy julio ahu...
2,146705,buenos días agradeceré su colaboraci...
3,72603,por favor s...
4,131404,junto con saludar aún no tengo ...
...,...,...
60479,165002,junto con saludar favor su ay...
60480,122748,buen día favor su ayuda para verificar en si...
60481,165002,favor generar usuario y contras...
60482,129127,buenas tardes favor habilitar equip...


In [8]:
X_train, X_test, y_train, y_test  = train_test_split(df_tickets["fullDESCRIPTION"], df_tickets["ownerid"], test_size = 0.2, random_state = 1)

In [9]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words=stopwords.words('spanish'))),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=15, random_state=42)),
                        ])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.5947420634920635

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3),
                 }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)

In [ ]:
gs_clf_svm.best_score_

In [ ]:
gs_clf_svm.best_params_

In [ ]:
counts = pd.Series(''.join(df_tickets.fullDESCRIPTION).split()).value_counts()
counts

In [ ]:
high_freq = counts[:int(pd.Series(''.join(df_tickets.fullDESCRIPTION).split()).count()*1/100)]
high_freq

In [7]:
df_tickets.loc[25000]['fullDESCRIPTION']

'      por favor dejar activas las claves y accesos de andrés ruiz  ya que se extiende su contrato hasta fin de año           desde ya muchas gracias           saludos    patricia         enviado desde  outlook mobile              de   raul adonio rubilar gonzalez  lt raul rubilar saesa cl gt    enviado   viernes  15 de mayo de 2020 3 09 p  m    para   camila torres  raul andres ramis mardones   cc   patricia julieta llanquilef antilen  rodrigo andres navarrete seguel  valeria alejandra vergara ruiz  sandra gema gonzalez gonzalez   asunto   re  vencimiento contrato est            estimada camila  de acuerdo a lo conversado y solicitado y de acuerdo a lo informado por nuestro sg raúl ramis el día de hoy viernes 15 05  el contrato del est se extiende hasta fin de año       saludos     raúl rubilar gonzález      jefe de area distribución     gerencia explotación   zonal los ríos     63 2276155    56 9   63004674        www gruposaesa cl                   de     patricia julieta llanquilef